# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [32]:
# YOUR CHANGES HERE
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def compute_qT_once(R, P, gamma, v):
    return R + gamma * P @ v

In [33]:
def iterate_values_once(R, P, gamma, v):
    return np.max(compute_qT_once(R, P, gamma, v), axis=0)

In [34]:
def value_iteration(R, P, gamma, max_iterations=100, tolerance=0.001):
    # initial approximation v_0
    v_old = np.zeros(R.shape[-1])

    for i in range(max_iterations):
        # compute v_{i+1}
        v_new = iterate_values_once(R, P, gamma, v_old)

        # check if values did not change much
        if np.max(np.abs(v_new - v_old)) < tolerance:
            return v_new

        v_old = v_new

    # return v_{max_iterations}
    return v_old

In [35]:
# from example code for reference
#gamma=0.9
#v_star = value_iteration(R, P, gamma)
#v_star

In [36]:
# set params
gamma = 1
do_nothing_aliases = {"do-nothing"}

# load files
T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
Rraw = pd.read_csv("twizzleflu-rewards.tsv",    sep="\t")
T.columns = [c.lower() for c in T.columns]
Rraw.columns = [c.lower() for c in Rraw.columns]

# Build state/action sets
states = sorted(set(T["state"]) | set(T["next_state"]))
actions = sorted(set(T["action"]) | set(Rraw["action"]))
s2i = {s:i for i,s in enumerate(states)}
a2i = {a:i for i,a in enumerate(actions)}
S, A = len(states), len(actions)

# build transition tensor P: (A, S, S)
P = np.zeros((A, S, S), dtype=float)
for _, r in T.iterrows():
    a = a2i[r["action"]]; s = s2i[r["state"]]; sp = s2i[r["next_state"]]
    P[a, s, sp] += float(r["probability"])

# build reward matrix R: (A, S)
if {"state","action","reward"}.issubset(Rraw.columns):
    Rsa = Rraw[["state","action","reward"]].copy()
else:
    raise ValueError("Rewards file must have (state, action, reward).")

R = np.zeros((A, S), dtype=float); R[:] = np.nan
for _, r in Rsa.iterrows():
    a = a2i[r["action"]]; s = s2i[r["state"]]
    R[a, s] = float(r["reward"])
R = np.nan_to_num(R, nan=0.0)

# 'do nothing' action index
lower_map = {a.lower(): a for a in actions}
do_a_idx = None
for alias in do_nothing_aliases:
    if alias in lower_map:
        do_a_idx = a2i[lower_map[alias]]
        break
if do_a_idx is None:
    raise ValueError(f"No 'do nothing' action found. Seen actions: {actions}")

# restrict to the do-nothing action
R_pi = R[do_a_idx:do_a_idx+1, :]    
P_pi = P[do_a_idx:do_a_idx+1, :, :] 

# run value iteration
v_star_rewards = value_iteration(R_pi, P_pi, gamma=gamma)

# convert to discomfort (rewards are negative discomfort)
discomfort_per_state = -v_star_rewards

print("States (in order):", states)
print("Expected discomfort per state under 'Do Nothing' (discounted, gamma=1):")
for s_name, d in zip(states, discomfort_per_state):
    print(f"  {s_name:>20s}: {float(d): .6f}")

States (in order): ['exposed-1', 'exposed-2', 'exposed-3', 'recovered', 'symptoms-1', 'symptoms-2', 'symptoms-3']
Expected discomfort per state under 'Do Nothing' (discounted, gamma=1):
             exposed-1:  3.410978
             exposed-2:  4.264491
             exposed-3:  5.331327
             recovered: -0.000000
            symptoms-1:  6.664819
            symptoms-2:  4.999779
            symptoms-3:  1.666654


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [37]:
# YOUR CHANGES HERE
do_nothing_discomfort = pd.DataFrame({
    "state": states,
    "expected_discomfort": discomfort_per_state
})
do_nothing_discomfort.to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)

Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [38]:
# YOUR CHANGES HERE
# same as above here
gamma = 1.0

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
Rraw = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
T.columns = [c.lower() for c in T.columns]
Rraw.columns = [c.lower() for c in Rraw.columns]

states = sorted(set(T["state"]) | set(T["next_state"]))
actions = sorted(set(T["action"]) | set(Rraw["action"]))
s2i = {s:i for i,s in enumerate(states)}
a2i = {a:i for i,a in enumerate(actions)}
S, A = len(states), len(actions)

P = np.zeros((A, S, S), dtype=float)
for _, r in T.iterrows():
    a = a2i[r["action"]]
    s = s2i[r["state"]]
    sp = s2i[r["next_state"]]
    P[a, s, sp] += float(r["probability"])

if {"state","action","reward"}.issubset(Rraw.columns):
    Rsa = Rraw[["state","action","reward"]].copy()
else:
    raise ValueError("Rewards file must have (state, action, reward).")

R = np.zeros((A, S), dtype=float); R[:] = np.nan
for _, r in Rsa.iterrows():
    a = a2i[r["action"]]
    s = s2i[r["state"]]
    R[a, s] = float(r["reward"])
R = np.nan_to_num(R, nan=0.0)

# value iteration over all actions
v_star_rewards = value_iteration(R, P, gamma=gamma)

# get optimal greedy policy from Q(a,s)
Q_star = compute_qT_once(R, P, gamma, v_star_rewards)  
best_action_idx_per_state = np.argmax(Q_star, axis=0)   
optimal_actions = [actions[i] for i in best_action_idx_per_state]

# convert to expected discomfort (in case we need direct comparison to above at some point)
expected_discomfort_opt = -v_star_rewards

# print outcome
print("Optimal treatment plan (policy):")
for s, a in zip(states, optimal_actions):
    print(f"  State {s:>20s} -> Action: {a}")

Optimal treatment plan (policy):
  State            exposed-1 -> Action: sleep-8
  State            exposed-2 -> Action: sleep-8
  State            exposed-3 -> Action: sleep-8
  State            recovered -> Action: do-nothing
  State           symptoms-1 -> Action: drink-oj
  State           symptoms-2 -> Action: drink-oj
  State           symptoms-3 -> Action: drink-oj


Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [39]:
# YOUR CHANGES HERE
optimal_policy=pd.DataFrame({
    "state": states, 
    "action": optimal_actions
    })

optimal_policy.to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)

Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [40]:
# YOUR CHANGES HERE
# Expected discomforts were saved out above in calculating the optimal policy
print("Expected discomfort per state:")
for s, a in zip(states, expected_discomfort_opt):
    print(f"  State {s:>20s} -> Expected discomfort: {a}")

Expected discomfort per state:
  State            exposed-1 -> Expected discomfort: 0.7491647930150531
  State            exposed-2 -> Expected discomfort: 1.4986889192678159
  State            exposed-3 -> Expected discomfort: 2.9979439870336204
  State            recovered -> Expected discomfort: -0.0
  State           symptoms-1 -> Expected discomfort: 5.996778913019338
  State           symptoms-2 -> Expected discomfort: 4.499579858219914
  State           symptoms-3 -> Expected discomfort: 1.499973182439569


Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [41]:
# YOUR CHANGES HERE
expected_discomfort=pd.DataFrame({
    "state": states, 
    "expected_discomfort": expected_discomfort_opt
    })

expected_discomfort.to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [42]:
# YOUR CHANGES HERE
# same as above here
gamma = 1.0

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
Rraw = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")  # not used for Part 4 rewards; just to gather actions
T.columns = [c.lower() for c in T.columns]
Rraw.columns = [c.lower() for c in Rraw.columns]

states  = sorted(set(T["state"]) | set(T["next_state"]))
actions = sorted(set(T["action"]) | set(Rraw.get("action", pd.Series([], dtype=str))))
s2i = {s:i for i,s in enumerate(states)}
a2i = {a:i for i,a in enumerate(actions)}

S, A = len(states), len(actions)

P = np.zeros((A, S, S), dtype=float)
for _, r in T.iterrows():
    a = a2i[r["action"]]
    s = s2i[r["state"]]
    sp = s2i[r["next_state"]]
    P[a, s, sp] += float(r['probability'])

# assign sick to -1
sick_states = {"symptoms-1", "symptoms-2", "symptoms-3"}  
sick_mask   = np.array([s in sick_states for s in states], dtype=bool)
better_mask = ~sick_mask

R_duration = np.zeros((A, S), dtype=float)
R_duration[:, sick_mask]   = -1.0
R_duration[:, better_mask] =  0.0

# value_iteration minimizing sick time
v_star_rewards = value_iteration(R_duration, P, gamma=gamma, max_iterations=10000, tolerance=1e-8)

Q_star = compute_qT_once(R_duration, P, gamma, v_star_rewards)   # shape (A, S)
best_action_idx_per_state = np.argmax(Q_star, axis=0)
optimal_actions = [actions[i] for i in best_action_idx_per_state]

expected_sick_days = -v_star_rewards

print("\nExpected sick days per state:")
for s, d in zip(states, expected_sick_days):
    print(f"  {s:>20s} -> {float(d):.6f}")


Expected sick days per state:
             exposed-1 -> 1.250000
             exposed-2 -> 2.500000
             exposed-3 -> 5.000000
             recovered -> -0.000000
            symptoms-1 -> 10.000000
            symptoms-2 -> 6.666667
            symptoms-3 -> 3.333333


Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [43]:
# YOUR CHANGES HERE
duration_rewards_df = []

for a in actions:
    for s in states:
        reward = -1.0 if s in sick_states else 0.0
        duration_rewards_df.append({
            "state": s,
            "action": a,
            "reward": reward
        })

duration_rewards_df = pd.DataFrame(duration_rewards_df)
duration_rewards_df.to_csv("duration-rewards.tsv", sep="\t", index=False)

Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [ ]:
# YOUR CHANGES HERE
# same again
gamma = 0.999

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
Rraw = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
T.columns = [c.lower() for c in T.columns]
Rraw.columns = [c.lower() for c in Rraw.columns]

state_col  = "state"         
action_col = "action"        
prob_col   = "probability"   

# Build list of states and actions
states_from_T = set(T[state_col].unique())
states_from_R = set(Rraw["state"].unique()) if "state" in Rraw.columns else set()
states = sorted(states_from_T | states_from_R)
actions = sorted(T[action_col].unique())
s2i = {s:i for i,s in enumerate(states)}
a2i = {a:i for i,a in enumerate(actions)}
S, A = len(states), len(actions)

# Build transition tensor P: (A,S,S)
P = np.zeros((A, S, S), dtype=float)

if prob_col is not None and "next_state" in T.columns:
    for _, r in T.iterrows():
        a  = a2i[r[action_col]]
        s  = s2i[r[state_col]]
        sp = s2i[r["next_state"]]
        P[a, s, sp] += float(r[prob_col])
else:
    id_like = {state_col, action_col}
    candidate_cols = [c for c in T.columns if c not in id_like]
    for c in candidate_cols:
        if c not in s2i:
            states.append(c)
            s2i[c] = len(s2i)
    # rebuild structures
    states = sorted(states)
    s2i = {s:i for i,s in enumerate(states)}
    S = len(states)
    P = np.zeros((A, S, S), dtype=float)
    for _, r in T.iterrows():
        a = a2i[r[action_col]]
        s = s2i[r[state_col]]
        row_probs = np.zeros(S)
        for sp_name in candidate_cols:
            row_probs[s2i[sp_name]] = float(r[sp_name])
        # normalize if necessary
        total = row_probs.sum()
        if total > 0 and not np.isclose(total, 1.0, atol=1e-6):
            row_probs /= total
        P[a, s, :] = row_probs


# define reward
sick_states = {"symptoms-1", "symptoms-2", "symptoms-3"} 
sick_mask   = np.array([s in sick_states for s in states], dtype=bool)
better_mask = ~sick_mask

R_duration = np.zeros((A, S), dtype=float)
R_duration[:, sick_mask]   = -1.0
R_duration[:, better_mask] =  0.0

v_star_rewards = value_iteration(R_duration, P, gamma=gamma, max_iterations=20000, tolerance=1e-10)
Q_star = compute_qT_once(R_duration, P, gamma, v_star_rewards)
best_action_idx_per_state = np.argmax(Q_star, axis=0)
optimal_actions = [actions[i] for i in best_action_idx_per_state]

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.



In [45]:
# YOUR CHANGES HERE
minimum_duration_actions=pd.DataFrame({
    "state": states, 
    "action": optimal_actions
    })

minimum_duration_actions.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [46]:
# YOUR CHANGES HERE

expected_sick_days = -v_star_rewards
expected_sick_days

array([ 1.23922232,  2.48092556,  4.96681793, -0.        ,  9.94357944,
        6.64008788,  3.32557366])

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [47]:
# YOUR CHANGES HERE
minimum_duration_days=pd.DataFrame({
    "state": states, 
    "expected_sick_days": expected_sick_days
    })

minimum_duration_days.to_csv("minimum-duration-days.tsv", sep="\t", index=False)

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [ ]:
# YOUR CHANGES HERE
expected_sick_days = -v_star_rewards  

# build original discomfort reward R_orig(a,s) from Rraw
if {"state","action","next_state","reward"}.issubset(Rraw.columns):
    R_sas = {}
    for _, rr in Rraw.iterrows():
        s0, a0, sp0 = rr["state"], rr["action"], rr["next_state"]
        if s0 in s2i and a0 in a2i and sp0 in s2i:
            R_sas[(a2i[a0], s2i[s0], s2i[sp0])] = float(rr["reward"])
    R_orig = np.zeros((A, S), dtype=float)
    for ai in range(A):
        for si in range(S):
            rewards_vec = np.array([R_sas.get((ai, si, sp), 0.0) for sp in range(S)], float)
            R_orig[ai, si] = np.dot(P[ai, si, :], rewards_vec)
else:
    # (state, action, reward)
    R_orig = np.zeros((A, S), dtype=float)
    for _, rr in Rraw[["state","action","reward"]].iterrows():
        if rr["state"] in s2i and rr["action"] in a2i:
            R_orig[a2i[rr["action"]], s2i[rr["state"]]] = float(rr["reward"])

# evaluate the duration-optimal policy under the original rewards
pi_dur = best_action_idx_per_state  # action index per state you already computed
R_pi = np.array([R_orig[pi_dur[s], s] for s in range(S)], float)
P_pi = np.array([P[pi_dur[s], s, :] for s in range(S)], float)

V = np.zeros(S, float)
for _ in range(20000):
    V_new = R_pi + gamma * (P_pi @ V)
    if np.max(np.abs(V_new - V)) < 1e-8:
        V = V_new
        break
    V = V_new

speed_discomfort = -V

print(speed_discomfort)

[ 0.82614744  1.65394883  3.31120888 -0.          6.62904681  4.98283077
  1.66278683]


Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [49]:
# YOUR CHANGES HERE

pd.DataFrame({
    "state": states,
    "speed_discomfort": speed_discomfort,
    "minimize_discomfort": expected_discomfort_opt
}).to_csv("policy-comparison.tsv", sep="\t", index=False)

Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.